<a href="https://colab.research.google.com/github/jku-win-dke/SemAI/blob/main/UE04_Test_SPARQL_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UE04_Test: SPARQL Update

## Preparations

In [ ]:
# Install required packages
!pip install -q rdflib     # comment to avoid re-install with every re-run

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00


### Imports and Functions

We are re-using the sparql_select function.

In [ ]:
# Imports
import pandas as pd
import rdflib
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace


# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]
    rows.append(row)                    #     add the dictionary (row) to the list
  return pd.DataFrame(rows,columns=results.vars)
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns


In [ ]:
g = rdflib.Graph().parse(format="turtle",data="""
  @prefix : <http://example.org/> .""")

g.update("""
INSERT DATA {
  :jane  a :Human ;
    :friend :mary, :bob, :goofy ;
    foaf:knows :jane .
  :mary a foaf:Person .
  :bob a :Human .
  :goofy a :Dog .
}
""")

# Group **A**

## Data

In [ ]:
print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:bob a :Human .

:goofy a :Dog .

:jane a :Human ;
    :friend :bob,
        :goofy,
        :mary ;
    foaf:knows :jane .

:mary a foaf:Person .




## Task

Write a SPARQL Update request so that all humans are not classified as `:Human` but as `foaf:Person`.

## Solution

In [ ]:
g.update("""
DELETE { ?p a :Human. }
INSERT { ?p a foaf:Person. }
WHERE { ?p a :Human. }
""")

## Result

In [ ]:
print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:bob a foaf:Person .

:goofy a :Dog .

:jane a foaf:Person ;
    :friend :bob,
        :goofy,
        :mary ;
    foaf:knows :jane .

:mary a foaf:Person .




# Group **B**

## Data

In [ ]:
print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:bob a foaf:Person .

:goofy a :Dog .

:jane a foaf:Person ;
    :friend :bob,
        :goofy,
        :mary ;
    foaf:knows :jane .

:mary a foaf:Person .




## Task

Write a SPARQL Update Request to replace all `:friend` statements with a  `foaf:knows` statement.

## Solution

In [ ]:
g.update("""
DELETE { ?x :friend ?y . }
INSERT { ?x foaf:knows ?y . }
WHERE { ?x :friend ?y . }
""")

## Result

In [ ]:
print(g.serialize(format="turtle"))

@prefix : <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

:bob a foaf:Person .

:goofy a :Dog .

:jane a foaf:Person ;
    foaf:knows :bob,
        :goofy,
        :jane,
        :mary .

:mary a foaf:Person .


